In [1]:
import pandas as pd
import numpy as np
import pulp
import random

In [2]:
# Create the properties of the offices
officeData = pd.DataFrame( { 'officeID': np.arange(0, 12)})
officeData['roomID'] = officeData['officeID']//2
officeData['isLeft'] = officeData['officeID']%2
officeData = officeData.set_index('officeID')
NRooms = len(officeData['roomID'].unique())
print(NRooms)
officeData

6


,roomID,isLeft
officeID,,
0,0,0
1,0,1
2,1,0
3,1,1
4,2,0
5,2,1
6,3,0
7,3,1
8,4,0


In [3]:
persoData = pd.DataFrame( {'persoID' : np.arange(8)})
persoData['isTall'] = np.random.choice([0, 1], size=(8,1), p=[0.7, 0.3] )
persoData['service'] = np.random.randint(0, 3, (8,1) )
persoData = persoData.set_index('persoID')
NServ = len(persoData['service'].unique())
print('NServ : ', NServ)
persoData

NServ :  3


,isTall,service
persoID,,
0,1,1
1,1,0
2,1,1
3,0,1
4,1,1
5,1,0
6,0,1
7,0,2


In [4]:
#Creating variables
officeOccupancy = pulp.LpVariable.matrix("officeOccupancy"
                                         ,(list(persoData.index), list(officeData.index))
                                         ,cat='Binary')

In [5]:
#Creating delta variables
delta = pulp.LpVariable.matrix("delta"
                                ,(np.arange(NServ), np.arange(NRooms))
                                ,cat='Binary')


In [6]:
R = pd.pivot_table(officeData, values=['isLeft'], columns='roomID', index=officeData.index, aggfunc='count').fillna(0).astype(int).values
R

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [7]:
S = pd.pivot_table(persoData, values='isTall', columns='service', index=persoData.index, aggfunc='count').fillna(0).astype(int).values.T
S

array([[0, 1, 0, 0, 0, 1, 0, 0],
       [1, 0, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [51]:
Delta = np.dot(np.dot(S, officeOccupancy),R)
Delta

array([[ 1*officeOccupancy_1_0 + 1*officeOccupancy_1_1 + 1*officeOccupancy_5_0 + 1*officeOccupancy_5_1 + 0,
        1*officeOccupancy_1_2 + 1*officeOccupancy_1_3 + 1*officeOccupancy_5_2 + 1*officeOccupancy_5_3 + 0,
        1*officeOccupancy_1_4 + 1*officeOccupancy_1_5 + 1*officeOccupancy_5_4 + 1*officeOccupancy_5_5 + 0,
        1*officeOccupancy_1_6 + 1*officeOccupancy_1_7 + 1*officeOccupancy_5_6 + 1*officeOccupancy_5_7 + 0,
        1*officeOccupancy_1_8 + 1*officeOccupancy_1_9 + 1*officeOccupancy_5_8 + 1*officeOccupancy_5_9 + 0,
        1*officeOccupancy_1_10 + 1*officeOccupancy_1_11 + 1*officeOccupancy_5_10 + 1*officeOccupancy_5_11 + 0],
       [ 1*officeOccupancy_0_0 + 1*officeOccupancy_0_1 + 1*officeOccupancy_2_0 + 1*officeOccupancy_2_1 + 1*officeOccupancy_3_0 + 1*officeOccupancy_3_1 + 1*officeOccupancy_4_0 + 1*officeOccupancy_4_1 + 1*officeOccupancy_6_0 + 1*officeOccupancy_6_1 + 0,
        1*officeOccupancy_0_2 + 1*officeOccupancy_0_3 + 1*officeOccupancy_2_2 + 1*officeOccupancy_2_

In [52]:
#Creating delta variables
DeltaUp = pulp.LpVariable.matrix("DeltaUP"
                                ,(np.arange(NServ), np.arange(NRooms))
                                ,cat='Binary')

DeltaDown = pulp.LpVariable.matrix("DeltaDown"
                                ,(np.arange(NServ), np.arange(NRooms))
                                ,cat='Binary')


In [55]:
model = pulp.LpProblem("Office setting maximizing happyness", pulp.LpMaximize)


#Objective function : maximising the number of people seated (constrained anyway)
model += pulp.lpSum( officeOccupancy )
#[iPerso][iRoom].varValue for iPerso in persoData.index for iRoom in officeData.index )

#Each perso is set once
for s  in np.sum(officeOccupancy, axis=0) :
#    print(s)
    model += s <= 1
    
#Each room is set at least once
for s in np.sum(officeOccupancy, axis=1) :
#    print(s)
    model += s == 1
        
#Constraint of delta
for s in range( NServ) :
    for j in range( NRooms ) :
        model += delta[s][j] >= Delta[s][j]/len(persoData)
        model += delta[s][j] <= Delta[s][j]

In [56]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [58]:
#Print results
for iPerso in persoData.index :
    for iRoom in officeData.index : 
        if officeOccupancy[iPerso][iRoom].varValue   :
            print( iPerso, iRoom)

0 3
1 2
2 0
3 9
4 7
5 4
6 5
7 6


In [ ]:
j = 2
s = 1
answer = False
officesInRoom = officeData.loc[officeData['roomID']==2].index.values
print('officesInRoom : ',j,  officesInRoom)
for x in persoData[persoData['service']==s].index :
    print([ officeOccupancy[x, room].varValue for room in officeData.index ])
    print(x)

In [ ]:
np.sum(np.array([[1,1],[1,1]]))

In [ ]:
x = np.zeros((len(persoData), len(officeData)))
distrib = np.random.choice( list(range(len(officeData))), size=len(persoData), replace=False)

for i in range(len(x)) : 
    x[i][distrib[i]] =1
x

In [ ]:
Delta =np.dot(S, x)
print('S : ', S)
print('Delta : ', Delta)
Delta = np.dot(Delta, R)
print('R : ', R)
Delta